# 3.6.3 Multiple Linear Regression

The MASS library contains the Boston data set, which records medv (median house value) for 506 neighborhoods around Boston. We will seek to predict medv using 13 predictors such as rm (average number of rooms per house), age (average age of houses), and lstat (percent of households with low socioeconomic status).

In [5]:
# %load ../dss_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('display.notebook_repr_html', False)

%matplotlib inline
plt.style.use('seaborn-white')

### Import Dataset

In [6]:
boston = pd.read_csv('Data/Boston.csv', usecols=list(range(1, 15)))
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
crim       506 non-null float64
zn         506 non-null float64
indus      506 non-null float64
chas       506 non-null int64
nox        506 non-null float64
rm         506 non-null float64
age        506 non-null float64
dis        506 non-null float64
rad        506 non-null int64
tax        506 non-null int64
ptratio    506 non-null float64
black      506 non-null float64
lstat      506 non-null float64
medv       506 non-null float64
dtypes: float64(11), int64(3)
memory usage: 55.4 KB


### Multiple Linear Regression

In order to fit a multiple linear regression model using least squares, we use the from\_formula() method. The syntax from\_formula(y~x1+x2+x3) is used to fit a model with three predictors, x1, x2, and x3. 


In [7]:
model = sm.OLS.from_formula('medv ~ lstat + age', boston)
lm_fit = model.fit()

The summary() function now outputs the regression coefficients for all the predictors.

In [8]:
print(lm_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     309.0
Date:                Mon, 10 Apr 2017   Prob (F-statistic):           2.98e-88
Time:                        23:26:45   Log-Likelihood:                -1637.5
No. Observations:                 506   AIC:                             3281.
Df Residuals:                     503   BIC:                             3294.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     33.2228      0.731     45.458      0.0

The Boston data set contains 13 variables, and so it would be cumbersome to have to type all of these in order to perform a regression using all of the predictors. Instead, we can use the following short-hand:

In [9]:
model = sm.OLS.from_formula('medv ~ ' + '+'.join(boston.columns.difference(['medv'])), boston)
lm_fit = model.fit()
print(lm_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 10 Apr 2017   Prob (F-statistic):          6.72e-135
Time:                        23:26:46   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

We can use the same approach to perform a regression using just a subset of the predictors. For example, in the above regression output, age have a high p-value. Because of this we may wish to run a regression excluding this predictor as follows:

In [10]:
model = sm.OLS.from_formula('medv ~ ' + '+'.join(boston.columns.difference(['medv', 'age'])), boston)
lm_fit = model.fit()
print(lm_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     117.3
Date:                Mon, 10 Apr 2017   Prob (F-statistic):          6.08e-136
Time:                        23:26:48   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3024.
Df Residuals:                     493   BIC:                             3079.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     36.4369      5.080      7.172      0.0

In the above regression output, indus have a high p-value. Because of this we may wish to run a regression excluding both age and indus as follows:

In [11]:
model = sm.OLS.from_formula('medv ~ ' + '+'.join(boston.columns.difference(['medv', 'age', 'indus'])), boston)
lm_fit = model.fit()
print(lm_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     128.2
Date:                Mon, 10 Apr 2017   Prob (F-statistic):          5.54e-137
Time:                        23:26:49   Log-Likelihood:                -1498.9
No. Observations:                 506   AIC:                             3022.
Df Residuals:                     494   BIC:                             3072.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     36.3411      5.067      7.171      0.0

### Variance Inflation Factor

Most VIF's are low to moderate for this data

In [23]:
# Break into left and right hand side; y and X
y, X = dmatrices("medv ~ crim + zn + indus + chas + nox + rm + age + dis + rad + tax + ptratio + black + lstat", data=boston, return_type="dataframe")

# For each Xi, calculate VIF
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Fit X to y
result = sm.OLS(y, X).fit()

print(vif)

[585.26523794231207, 1.7921915474332406, 2.2987581787494409, 3.9915964183460333, 1.0739953275537886, 4.3937198475774952, 1.9337444357832565, 3.1008255128153364, 3.9559449063727281, 7.4844963352744722, 9.0085539475970702, 1.7990840492488984, 1.3485210764063758, 2.9414910780919366]
